# Notebook with a numerical example on the computation of $\Theta$ and $\Omega$ Yukawas from neutrino one-loop formulas

In [1]:
import numpy as np
from scipy import optimize
import random

#printing for arrays. To offer a better visualization in the notebook
np.set_printoptions(linewidth=np.inf)

# Helpful functions

In [2]:
def eigen(A):
    
    """
    Computes the eigenvalues and eigenvectors, and orders the output
    from smallest to hightest.
    
    Returns a tuple with the eigenvalues (first position) and eigenvectors
    (second position)
    
    Example use: Out_eigvalues, Output_eigvectors = eigen(Matrix)
    
    """
    
    eigenValues, eigenVectors = np.linalg.eig(A.conj().T @ A)
    idx = np.argsort(np.abs(eigenValues))
    eigenValues = np.sqrt(np.abs(eigenValues[idx]))
    eigenVectors = eigenVectors[:,idx].T
    np.abs(eigenVectors)

    return (eigenValues, eigenVectors)

def pmlogscan(min,max):
    
    """
    Random logarithmic scan. Requires the inputs
    
      -> min: Minimum value that the random number can take.
              Note that it can not be zero, since log(0) = Inf
      -> max: Maximum value that the random number can take
    
    Generates both positive or negative numbers
    
    Example use: A = pmlogscan(1e-8, 4)
    """
    
    signal = random.choice(np.array([-1,1]))
    
    low_index = np.log10(min)
    high_index = np.log10(max) 
    power = np.random.uniform(low_index , high_index) 
    
    value = signal * 10**power
    
    return value 

def logscan(min,max):

    """
    Random logarithmic scan. Requires the inputs
    
      -> min: Minimum value that the random number can take.
              Note that it can not be zero, since log(0) = Inf
      -> max: Maximum value that the random number can take
    
    Generates only positive numbers
    
    Example use: A = logscan(1e-8, 4)
    """
    
    low_index= np.log10(min)
    high_index=np.log10(max) 
    power = np.random.uniform( low_index , high_index ) 
    
    value = 10**power
    
    return value 

def linscan(min,max):

    """
    Random linear scan. Requires the inputs
    
      -> min: Minimum value that the random number can take.
      -> max: Maximum value that the random number can take
    
    Generates only positive numbers
    
    Example use: A = linscan(1e-8, 4)
    """
    
    val = np.random.uniform( min , max ) 
    
    return  val

def pmlinscan(min,max):

    """
    Random linear scan. Requires the inputs
    
      -> min: Minimum value that the random number can take.
      -> max: Maximum value that the random number can take
    
    Generates only positive numbers
    
    Example use: A = pmlinscan(1e-8, 4)
    """
    
    signal = random.choice(np.array([-1,1]))
    
    val = np.random.uniform( min , max ) 
    
    output = signal * val
    
    return  output

def PMNS(a13, a23, a12, dCP):
    
    """
    Determine the experimental PMNS matrix, based on the angles. PDG parametrization
    
    Inputs:
        -> a13: Angle between 1 and 3 lepton states
        -> a23: Angle between 2 and 2 lepton states
        -> a12: Angle between 1 and 2 lepton states
        -> dCP: CP violating phase
        
    There is numerical errors when picking dCP = 180. It prints out a non-zero imaginary part. It is very small,
    of the order O(1e-16), but causes errors when trying to find solutions
    A quick hack is added to make Exp(-i*180º) = -1
    
    Additional info: https://stackoverflow.com/questions/59021598/numpy-precision-with-exponentials-of-imaginary-numbers
    
    Example use: UPMNS = PMNS(a13 = 33.45, a12 = 42.1, a13 = 8.62, dCP = 230)
                 or
                 UPMNS = PMNS(33.45, 42.1, 8.62, 230)
    """

    if dCP == 180:
        
        ad13 = np.deg2rad(a13)
        ad23 = np.deg2rad(a23)
        ad12 = np.deg2rad(a12)
        ddCP = np.deg2rad(dCP)
        Exp1 = np.round(np.exp(-1j*ddCP))
        Exp2 = np.round(np.exp(+1j*ddCP))


        R1 = np.array([[1,0,0], [0,np.cos(ad23),np.sin(ad23)], [0,-np.sin(ad23),np.cos(ad23)]])
        R2 = np.array([[np.cos(ad13), 0, np.sin(ad13)*Exp1], [0,1,0], [-np.sin(ad13)*Exp2, 0, np.cos(ad13)]])
        R3 = np.array([[np.cos(ad12), np.sin(ad12), 0], [-np.sin(ad12), np.cos(ad12), 0], [0,0,1]])
    
    else:
        
        ad13 = np.deg2rad(a13)
        ad23 = np.deg2rad(a23)
        ad12 = np.deg2rad(a12)
        ddCP = np.deg2rad(dCP)

        R1 = np.array([[1,0,0], [0,np.cos(ad23),np.sin(ad23)], [0,-np.sin(ad23),np.cos(ad23)]])
        R2 = np.array([[np.cos(ad13), 0, np.sin(ad13)*np.exp(-1j*ddCP)], [0,1,0], [-np.sin(ad13)*np.exp(1j*ddCP), 0, np.cos(ad13)]])
        R3 = np.array([[np.cos(ad12), np.sin(ad12), 0], [-np.sin(ad12), np.cos(ad12), 0], [0,0,1]])
        
    Vpmns = R1 @ R2 @ R3
    
    return Vpmns

def CKM (s12, s13, s23, d):

    """
    Determine the experimental CKM matrix, based on the standard parameterization
    
    Inputs:
        -> s12: sine of the angle between 1 and 2 quark states
        -> s13: sine of the angle between 1 and 3 quark states
        -> s23: sine of the angle between 2 and 3 quark states
        -> d: CP violating phase
        
    Example use: VCKM = VCKM(s12 = 0.22650, s13 = {}0361, s23 = {}4053, d = 1.196)
                 or
                 VCKM = VCKM(0.22650, {}0361, {}4053, 1.196)
    """
    
    c12 = np.sqrt(1 - s12**2)
    c13 = np.sqrt(1 - s13**2)
    c23 = np.sqrt(1 - s23**2)
    
    R1 = np.array([[1,0,0],[0,c23,s23],[0,-s23,c23]])
    R2 = np.array([[c13,0,s13*np.exp(-1j*d)],[0,1,0],[-s13*np.exp(1j*d),0,c13]])
    R3 = np.array([[c12,s12,0],[-s12,c12,0],[0,0,1]])
    
    Vckm = R1 @ R2 @ R3
    
    return Vckm

#Neutrino one-loop mass formula. Corresponds to equation (5) in the main text
def Neutrino_oneloop(i, j, Omega, Theta, VCKM, v, a1, md, mS1, mS2):
    
    """
    Computes the loop function for the neutrino mass matrix. Takes as input
    the Yukawa matrices, Omega and Theta, the CKM mixing matrix, the cubic term
    a1, the vaccuum expectation value, the leptoquark masses (mS1 and mS2) and the
    down-type quark masses.
    
    Also requires the matrix element position (i,j)
    
    """
    
    Cte = (3*alpha1*v)/(16*np.sqrt(2)*(mS2**2 - mS1**2)*np.pi**2)*np.log(mS2**2/mS1**2)
    
    F = Cte*sum(md[a]*VCKM[a,m]*(Theta[i][m]*Omega[j][a] + Theta[j][m]*Omega[i][a]) for m in range(3) for a in range(3)) 
    
    return F

#Leptoquark inversion formula. Necessary in scan to give leptoquark masses as input parameters
#See, for example, in 2105.08670. Corresponds to the inversion of the equations (6)
def Leptoquark_inversion(mS1, mS2, lHR, lHRp, alpha1, lHS, v):
    
    """
    Determines the muR and muS parameters
    
    Inputs:
        -> mS1: Physical mass of the first S leptoquark
        -> mS2: Physical mass of the second S leptoquark
        -> lHS: Quartic coupling H\H\S\S
        -> alpha1: Triple coupling R\S\H
        -> lHRp: Quartic coupling H\R\R\H
        -> lHR: Quartic coupling H\H\R\R
        
    """
    
    muS = (1/2)*(mS1**2 + mS2**2 - lHS*(v**2) - np.sqrt((mS1**2 - mS2**2)**2 - 2*(alpha1**2)*(v**2)))
    muR = (1/2)*(mS1**2 + mS2**2 - (lHR + lHRp)*(v**2) + np.sqrt((mS1**2 - mS2**2)**2 - 2*(alpha1**2)*(v**2)))
    
    return muR, muS


# Inverted one-loop mass equations. scipy.root function only works in the real domain, so we need to separate the equations into imaginary and real parts 

# Here I choose such that all $\Theta$'s are free inputs and all the $\Omega$'s are calculated from the inversion. However, it is only a choice and it can be alterated to any other combination

In [3]:
def Equations_neutrinos(x):
        
    """
    Diagonalisation of the neutrinos:
        ---> UPMNS^\dagger * Mnu_diag * UPMNS = Mnu
    
    """
    
    ##
    Cte = (3*alpha1*v)/(16*np.sqrt(2)*(mS2**2 - mS1**2)*np.pi**2)*np.log(mS2**2/mS1**2)
    Vreal = VCKM.real
    Vimag = VCKM.imag
    ##
    
    ##
    Mnu_diag = np.diag(np.array([m1p, m2p, m3p]))
    MN = VPMNS.conj().T @ Mnu_diag @ VPMNS
    MN_real = MN.real
    MN_imag = MN.imag
    ##
    
    ##
    Omegar = np.array([[x[0], x[1], x[2]], [x[3], x[4], x[5]], [x[6], x[7], x[8]]])
    Omegaim = np.array([[x[9], x[10], x[11]], [x[12], x[13], x[14]], [x[15], x[16], x[17]]])
    ##
    
    #Real equations
    i,j = 0,0
    EqR1 = Cte*sum(-md[a]*Omegar[j, a]*Thetaim[i, m]*Vimag[a, m] - md[a]*Omegar[i, a]*Thetaim[j, m]*Vimag[a, m] - md[a]*Omegaim[j, a]*Thetar[i, m]*Vimag[a, m] - \
                    md[a]*Omegaim[i, a]*Thetar[j, m]*Vimag[a, m] - md[a]*Omegaim[j, a]*Thetaim[i, m]*Vreal[a, m] - md[a]*Omegaim[i, a]*Thetaim[j, m]*Vreal[a, m] + \
                    md[a]*Omegar[j, a]*Thetar[i, m]*Vreal[a, m] + md[a]*Omegar[i, a]*Thetar[j, m]*Vreal[a, m] for m in range(3) for a in range(3)) + MN_real[i, j]    
    
    i,j = 0,1
    EqR2 = Cte*sum(-md[a]*Omegar[j, a]*Thetaim[i, m]*Vimag[a, m] - md[a]*Omegar[i, a]*Thetaim[j, m]*Vimag[a, m] - md[a]*Omegaim[j, a]*Thetar[i, m]*Vimag[a, m] - \
                    md[a]*Omegaim[i, a]*Thetar[j, m]*Vimag[a, m] - md[a]*Omegaim[j, a]*Thetaim[i, m]*Vreal[a, m] - md[a]*Omegaim[i, a]*Thetaim[j, m]*Vreal[a, m] + \
                    md[a]*Omegar[j, a]*Thetar[i, m]*Vreal[a, m] + md[a]*Omegar[i, a]*Thetar[j, m]*Vreal[a, m] for m in range(3) for a in range(3)) + MN_real[i, j]     
    
    i,j = 0,2
    EqR3 = Cte*sum(-md[a]*Omegar[j, a]*Thetaim[i, m]*Vimag[a, m] - md[a]*Omegar[i, a]*Thetaim[j, m]*Vimag[a, m] - md[a]*Omegaim[j, a]*Thetar[i, m]*Vimag[a, m] - \
                    md[a]*Omegaim[i, a]*Thetar[j, m]*Vimag[a, m] - md[a]*Omegaim[j, a]*Thetaim[i, m]*Vreal[a, m] - md[a]*Omegaim[i, a]*Thetaim[j, m]*Vreal[a, m] + \
                    md[a]*Omegar[j, a]*Thetar[i, m]*Vreal[a, m] + md[a]*Omegar[i, a]*Thetar[j, m]*Vreal[a, m] for m in range(3) for a in range(3)) + MN_real[i, j] 
    
    i,j = 1,0
    EqR4 = Cte*sum(-md[a]*Omegar[j, a]*Thetaim[i, m]*Vimag[a, m] - md[a]*Omegar[i, a]*Thetaim[j, m]*Vimag[a, m] - md[a]*Omegaim[j, a]*Thetar[i, m]*Vimag[a, m] - \
                    md[a]*Omegaim[i, a]*Thetar[j, m]*Vimag[a, m] - md[a]*Omegaim[j, a]*Thetaim[i, m]*Vreal[a, m] - md[a]*Omegaim[i, a]*Thetaim[j, m]*Vreal[a, m] + \
                    md[a]*Omegar[j, a]*Thetar[i, m]*Vreal[a, m] + md[a]*Omegar[i, a]*Thetar[j, m]*Vreal[a, m] for m in range(3) for a in range(3)) + MN_real[i, j] 
    
    i,j = 1,1
    EqR5 = Cte*sum(-md[a]*Omegar[j, a]*Thetaim[i, m]*Vimag[a, m] - md[a]*Omegar[i, a]*Thetaim[j, m]*Vimag[a, m] - md[a]*Omegaim[j, a]*Thetar[i, m]*Vimag[a, m] - \
                    md[a]*Omegaim[i, a]*Thetar[j, m]*Vimag[a, m] - md[a]*Omegaim[j, a]*Thetaim[i, m]*Vreal[a, m] - md[a]*Omegaim[i, a]*Thetaim[j, m]*Vreal[a, m] + \
                    md[a]*Omegar[j, a]*Thetar[i, m]*Vreal[a, m] + md[a]*Omegar[i, a]*Thetar[j, m]*Vreal[a, m] for m in range(3) for a in range(3)) + MN_real[i, j] 
    
    i,j = 1,2
    EqR6 = Cte*sum(-md[a]*Omegar[j, a]*Thetaim[i, m]*Vimag[a, m] - md[a]*Omegar[i, a]*Thetaim[j, m]*Vimag[a, m] - md[a]*Omegaim[j, a]*Thetar[i, m]*Vimag[a, m] - \
                    md[a]*Omegaim[i, a]*Thetar[j, m]*Vimag[a, m] - md[a]*Omegaim[j, a]*Thetaim[i, m]*Vreal[a, m] - md[a]*Omegaim[i, a]*Thetaim[j, m]*Vreal[a, m] + \
                    md[a]*Omegar[j, a]*Thetar[i, m]*Vreal[a, m] + md[a]*Omegar[i, a]*Thetar[j, m]*Vreal[a, m] for m in range(3) for a in range(3)) + MN_real[i, j] 
    
    i,j = 2,0
    EqR7 = Cte*sum(-md[a]*Omegar[j, a]*Thetaim[i, m]*Vimag[a, m] - md[a]*Omegar[i, a]*Thetaim[j, m]*Vimag[a, m] - md[a]*Omegaim[j, a]*Thetar[i, m]*Vimag[a, m] - \
                    md[a]*Omegaim[i, a]*Thetar[j, m]*Vimag[a, m] - md[a]*Omegaim[j, a]*Thetaim[i, m]*Vreal[a, m] - md[a]*Omegaim[i, a]*Thetaim[j, m]*Vreal[a, m] + \
                    md[a]*Omegar[j, a]*Thetar[i, m]*Vreal[a, m] + md[a]*Omegar[i, a]*Thetar[j, m]*Vreal[a, m] for m in range(3) for a in range(3)) + MN_real[i, j] 
        
    i,j = 2,1
    EqR8 = Cte*sum(-md[a]*Omegar[j, a]*Thetaim[i, m]*Vimag[a, m] - md[a]*Omegar[i, a]*Thetaim[j, m]*Vimag[a, m] - md[a]*Omegaim[j, a]*Thetar[i, m]*Vimag[a, m] - \
                    md[a]*Omegaim[i, a]*Thetar[j, m]*Vimag[a, m] - md[a]*Omegaim[j, a]*Thetaim[i, m]*Vreal[a, m] - md[a]*Omegaim[i, a]*Thetaim[j, m]*Vreal[a, m] + \
                    md[a]*Omegar[j, a]*Thetar[i, m]*Vreal[a, m] + md[a]*Omegar[i, a]*Thetar[j, m]*Vreal[a, m] for m in range(3) for a in range(3)) + MN_real[i, j] 
    
    i,j = 2,2
    EqR9 = Cte*sum(-md[a]*Omegar[j, a]*Thetaim[i, m]*Vimag[a, m] - md[a]*Omegar[i, a]*Thetaim[j, m]*Vimag[a, m] - md[a]*Omegaim[j, a]*Thetar[i, m]*Vimag[a, m] - \
                    md[a]*Omegaim[i, a]*Thetar[j, m]*Vimag[a, m] - md[a]*Omegaim[j, a]*Thetaim[i, m]*Vreal[a, m] - md[a]*Omegaim[i, a]*Thetaim[j, m]*Vreal[a, m] + \
                    md[a]*Omegar[j, a]*Thetar[i, m]*Vreal[a, m] + md[a]*Omegar[i, a]*Thetar[j, m]*Vreal[a, m] for m in range(3) for a in range(3)) + MN_real[i, j]     
    
    
    #Imaginary equations
    i,j = 0,0
    EqIm1 = Cte*sum(-md[a]*Omegaim[j, a]*Thetaim[i, m]*Vimag[a, m] - md[a]*Omegaim[i, a]*Thetaim[j, m]*Vimag[a, m] + md[a]*Omegar[j, a]*Thetar[i, m]*Vimag[a, m] + \
                     md[a]*Omegar[i, a]*Thetar[j, m]*Vimag[a, m] + md[a]*Omegar[j, a]*Thetaim[i, m]*Vreal[a, m] + md[a]*Omegar[i, a]*Thetaim[j, m]*Vreal[a, m] + \
                     md[a]*Omegaim[j, a]*Thetar[i, m]*Vreal[a, m] + md[a]*Omegaim[i, a]*Thetar[j, m]*Vreal[a, m] for m in range(3) for a in range(3)) + MN_imag[i, j]  
    
    i,j = 0,1
    EqIm2 = Cte*sum(-md[a]*Omegaim[j, a]*Thetaim[i, m]*Vimag[a, m] - md[a]*Omegaim[i, a]*Thetaim[j, m]*Vimag[a, m] + md[a]*Omegar[j, a]*Thetar[i, m]*Vimag[a, m] + \
                     md[a]*Omegar[i, a]*Thetar[j, m]*Vimag[a, m] + md[a]*Omegar[j, a]*Thetaim[i, m]*Vreal[a, m] + md[a]*Omegar[i, a]*Thetaim[j, m]*Vreal[a, m] + \
                     md[a]*Omegaim[j, a]*Thetar[i, m]*Vreal[a, m] + md[a]*Omegaim[i, a]*Thetar[j, m]*Vreal[a, m] for m in range(3) for a in range(3)) + MN_imag[i, j]
    
    i,j = 0,2
    EqIm3 = Cte*sum(-md[a]*Omegaim[j, a]*Thetaim[i, m]*Vimag[a, m] - md[a]*Omegaim[i, a]*Thetaim[j, m]*Vimag[a, m] + md[a]*Omegar[j, a]*Thetar[i, m]*Vimag[a, m] + \
                     md[a]*Omegar[i, a]*Thetar[j, m]*Vimag[a, m] + md[a]*Omegar[j, a]*Thetaim[i, m]*Vreal[a, m] + md[a]*Omegar[i, a]*Thetaim[j, m]*Vreal[a, m] + \
                     md[a]*Omegaim[j, a]*Thetar[i, m]*Vreal[a, m] + md[a]*Omegaim[i, a]*Thetar[j, m]*Vreal[a, m] for m in range(3) for a in range(3)) + MN_imag[i, j]
    
    i,j = 1,0
    EqIm4 = Cte*sum(-md[a]*Omegaim[j, a]*Thetaim[i, m]*Vimag[a, m] - md[a]*Omegaim[i, a]*Thetaim[j, m]*Vimag[a, m] + md[a]*Omegar[j, a]*Thetar[i, m]*Vimag[a, m] + \
                     md[a]*Omegar[i, a]*Thetar[j, m]*Vimag[a, m] + md[a]*Omegar[j, a]*Thetaim[i, m]*Vreal[a, m] + md[a]*Omegar[i, a]*Thetaim[j, m]*Vreal[a, m] + \
                     md[a]*Omegaim[j, a]*Thetar[i, m]*Vreal[a, m] + md[a]*Omegaim[i, a]*Thetar[j, m]*Vreal[a, m] for m in range(3) for a in range(3)) + MN_imag[i, j]
    
    i,j = 1,1
    EqIm5 = Cte*sum(-md[a]*Omegaim[j, a]*Thetaim[i, m]*Vimag[a, m] - md[a]*Omegaim[i, a]*Thetaim[j, m]*Vimag[a, m] + md[a]*Omegar[j, a]*Thetar[i, m]*Vimag[a, m] + \
                     md[a]*Omegar[i, a]*Thetar[j, m]*Vimag[a, m] + md[a]*Omegar[j, a]*Thetaim[i, m]*Vreal[a, m] + md[a]*Omegar[i, a]*Thetaim[j, m]*Vreal[a, m] + \
                     md[a]*Omegaim[j, a]*Thetar[i, m]*Vreal[a, m] + md[a]*Omegaim[i, a]*Thetar[j, m]*Vreal[a, m] for m in range(3) for a in range(3)) + MN_imag[i, j]
    
    i,j = 1,2
    EqIm6 = Cte*sum(-md[a]*Omegaim[j, a]*Thetaim[i, m]*Vimag[a, m] - md[a]*Omegaim[i, a]*Thetaim[j, m]*Vimag[a, m] + md[a]*Omegar[j, a]*Thetar[i, m]*Vimag[a, m] + \
                     md[a]*Omegar[i, a]*Thetar[j, m]*Vimag[a, m] + md[a]*Omegar[j, a]*Thetaim[i, m]*Vreal[a, m] + md[a]*Omegar[i, a]*Thetaim[j, m]*Vreal[a, m] + \
                     md[a]*Omegaim[j, a]*Thetar[i, m]*Vreal[a, m] + md[a]*Omegaim[i, a]*Thetar[j, m]*Vreal[a, m] for m in range(3) for a in range(3)) + MN_imag[i, j]
    
    i,j = 2,0
    EqIm7 = Cte*sum(-md[a]*Omegaim[j, a]*Thetaim[i, m]*Vimag[a, m] - md[a]*Omegaim[i, a]*Thetaim[j, m]*Vimag[a, m] + md[a]*Omegar[j, a]*Thetar[i, m]*Vimag[a, m] + \
                     md[a]*Omegar[i, a]*Thetar[j, m]*Vimag[a, m] + md[a]*Omegar[j, a]*Thetaim[i, m]*Vreal[a, m] + md[a]*Omegar[i, a]*Thetaim[j, m]*Vreal[a, m] + \
                     md[a]*Omegaim[j, a]*Thetar[i, m]*Vreal[a, m] + md[a]*Omegaim[i, a]*Thetar[j, m]*Vreal[a, m] for m in range(3) for a in range(3)) + MN_imag[i, j]
    
    i,j = 2,1
    EqIm8 = Cte*sum(-md[a]*Omegaim[j, a]*Thetaim[i, m]*Vimag[a, m] - md[a]*Omegaim[i, a]*Thetaim[j, m]*Vimag[a, m] + md[a]*Omegar[j, a]*Thetar[i, m]*Vimag[a, m] + \
                     md[a]*Omegar[i, a]*Thetar[j, m]*Vimag[a, m] + md[a]*Omegar[j, a]*Thetaim[i, m]*Vreal[a, m] + md[a]*Omegar[i, a]*Thetaim[j, m]*Vreal[a, m] + \
                     md[a]*Omegaim[j, a]*Thetar[i, m]*Vreal[a, m] + md[a]*Omegaim[i, a]*Thetar[j, m]*Vreal[a, m] for m in range(3) for a in range(3)) + MN_imag[i, j]
    
    i,j = 2,2
    EqIm9 = Cte*sum(-md[a]*Omegaim[j, a]*Thetaim[i, m]*Vimag[a, m] - md[a]*Omegaim[i, a]*Thetaim[j, m]*Vimag[a, m] + md[a]*Omegar[j, a]*Thetar[i, m]*Vimag[a, m] + \
                     md[a]*Omegar[i, a]*Thetar[j, m]*Vimag[a, m] + md[a]*Omegar[j, a]*Thetaim[i, m]*Vreal[a, m] + md[a]*Omegar[i, a]*Thetaim[j, m]*Vreal[a, m] + \
                     md[a]*Omegaim[j, a]*Thetar[i, m]*Vreal[a, m] + md[a]*Omegaim[i, a]*Thetar[j, m]*Vreal[a, m] for m in range(3) for a in range(3)) + MN_imag[i, j]
    
        
    f = [EqR1, EqR2, EqR3, EqR4, EqR5, EqR6, EqR7, EqR8, EqR9, EqIm1, EqIm2, EqIm3, EqIm4, EqIm5, EqIm6, EqIm7, EqIm8, EqIm9]
        
    return f    


# Determing the Yukawas with scipy

In [4]:
for i in range(0, int(1e9)):
    #SM parameters
    #Masses of the down in GeV (at the top scale)
    #See paper https://inspirehep.net/files/71930f37457ebd10742e55c2b36531ea

    mdown = linscan(2.56 - 2*0.18, 2.56 + 2*0.18)*1e-3
    mstrange = linscan(50.90 - 2*4.41, 50.90 + 2*4.41)*1e-3
    mbottom = linscan(2.702 - 2*0.025, 2.702 + 2*0.025)
    md = np.array([mdown, mstrange, mbottom])

    #VEV
    GF = linscan(1.1663787e-05 -2* 0.0000006e-05, 1.1663787e-05 +2* 0.0000006e-05)
    v  = np.sqrt((np.sqrt(2))/(2*GF))

    #CKM angles from latest PDG
    s12q = linscan(0.22650 - 2*0.00048, 0.22650 + 2*0.00048)
    s13q = linscan(0.00361 - 2*0.00011, 0.00361 + 2*0.00009)
    s23q = linscan(0.04053 - 2*0.00083, 0.04053 + 2*0.00061)
    DqCP = linscan(1.196 - 2*0.045, 1.196 + 2*0.043)
    VCKM = CKM(s12=s12q, s13=s13q, s23=s23q, d=DqCP)

    #Neutrino observables
    #Latest NUFIT
    #With SK atmospheric data. Normal ordering and neutrino mass differences in eV^2!!
    #Angles in degrees. Default in numpy is radians. Must convert (this is done inside the PMNS function)
    Theta12 = linscan(33.45 - 3*0.75, 33.45 + 3*0.77)
    Theta23 = linscan(42.1 - 3*0.9, 42.1 + 3*1.1)
    Theta13 = linscan(8.62 - 3*0.12, 8.62 + 3*0.12)
    deltaCP = linscan(230 - 3*25, 230 + 3*36)
    sdeltam21 = linscan(7.42 - 3*0.20, 7.42 + 3*0.21)*1e-5
    sdeltam32 = linscan(2.510 - 3*0.027, 2.510 + 3*0.027)*1e-3
    VPMNS = PMNS(a13 = Theta13, a23 = Theta23, a12 = Theta12, dCP = 180)
    #Neutrino masses
    m1 = logscan(1e-8, 0.6)
    m2 = np.sqrt(sdeltam21 + m1**2)
    m3 = np.sqrt(sdeltam32 + m2**2)

    m1p = m1*1e-9
    m2p = m2*1e-9
    m3p = m3*1e-9
    
    #New physics parameters
    mS1 = logscan(1500, 5000)
    mS2 = logscan(1500, 5000)
    alpha1 = logscan(1e-8, 100)
    
    #Solving for the equations. Theta as input and Omega as result of inversion
    Thetar11 = pmlogscan(1e-8, np.sqrt(4*np.pi))
    Thetar12 = pmlogscan(1e-8, np.sqrt(4*np.pi))
    Thetar13 = pmlogscan(1e-8, np.sqrt(4*np.pi))
    Thetar21 = pmlogscan(1e-8, np.sqrt(4*np.pi))
    Thetar22 = pmlogscan(1e-8, np.sqrt(4*np.pi))
    Thetar23 = pmlogscan(1e-8, np.sqrt(4*np.pi))
    Thetar31 = pmlogscan(1e-8, np.sqrt(4*np.pi))
    Thetar32 = pmlogscan(1e-8, np.sqrt(4*np.pi))
    Thetar33 = pmlogscan(1e-8, np.sqrt(4*np.pi))
    Thetar = np.array([[Thetar11, Thetar12, Thetar13],[Thetar21, Thetar22, Thetar23],[Thetar31, Thetar32, Thetar33]])

    Thetaim11 = pmlogscan(1e-8, np.sqrt(4*np.pi))
    Thetaim12 = pmlogscan(1e-8, np.sqrt(4*np.pi))
    Thetaim13 = pmlogscan(1e-8, np.sqrt(4*np.pi))
    Thetaim21 = pmlogscan(1e-8, np.sqrt(4*np.pi))
    Thetaim22 = pmlogscan(1e-8, np.sqrt(4*np.pi))
    Thetaim23 = pmlogscan(1e-8, np.sqrt(4*np.pi))
    Thetaim31 = pmlogscan(1e-8, np.sqrt(4*np.pi))
    Thetaim32 = pmlogscan(1e-8, np.sqrt(4*np.pi))
    Thetaim33 = pmlogscan(1e-8, np.sqrt(4*np.pi))
    Thetaim = np.array([[Thetaim11, Thetaim12, Thetaim13],[Thetaim21, Thetaim22, Thetaim23],[Thetaim31, Thetaim32, Thetaim33]])
    
    #scipy.root requires an initial guess to initialise the algorithm
    Omegar11_guess = pmlogscan(1e-8, np.sqrt(4*np.pi))
    Omegar12_guess = pmlogscan(1e-8, np.sqrt(4*np.pi))
    Omegar13_guess = pmlogscan(1e-8, np.sqrt(4*np.pi))
    Omegar21_guess = pmlogscan(1e-8, np.sqrt(4*np.pi))
    Omegar22_guess = pmlogscan(1e-8, np.sqrt(4*np.pi))
    Omegar23_guess = pmlogscan(1e-8, np.sqrt(4*np.pi))
    Omegar31_guess = pmlogscan(1e-8, np.sqrt(4*np.pi))
    Omegar32_guess = pmlogscan(1e-8, np.sqrt(4*np.pi))
    Omegar33_guess = pmlogscan(1e-8, np.sqrt(4*np.pi))

    Omegaim11_guess = pmlogscan(1e-8, np.sqrt(4*np.pi))
    Omegaim12_guess = pmlogscan(1e-8, np.sqrt(4*np.pi))
    Omegaim13_guess = pmlogscan(1e-8, np.sqrt(4*np.pi))
    Omegaim21_guess = pmlogscan(1e-8, np.sqrt(4*np.pi))
    Omegaim22_guess = pmlogscan(1e-8, np.sqrt(4*np.pi))
    Omegaim23_guess = pmlogscan(1e-8, np.sqrt(4*np.pi))
    Omegaim31_guess = pmlogscan(1e-8, np.sqrt(4*np.pi))
    Omegaim32_guess = pmlogscan(1e-8, np.sqrt(4*np.pi))
    Omegaim33_guess = pmlogscan(1e-8, np.sqrt(4*np.pi))

    #Solve equations
    solution = optimize.root(Equations_neutrinos, [Omegar11_guess, Omegar12_guess, Omegar13_guess, Omegar21_guess, Omegar22_guess, Omegar23_guess,
                                                   Omegar31_guess, Omegar32_guess, Omegar33_guess, Omegaim11_guess, Omegaim12_guess, Omegaim13_guess,
                                                   Omegaim21_guess, Omegaim22_guess, Omegaim23_guess, Omegaim31_guess, Omegaim32_guess, Omegaim33_guess],
                         method='hybr', options={'xtol': 1e-12})
    
    #If the algorithm converges and solutions are perturbative, stop the code.
    if solution.success == True:
        if (np.abs(solution.x[0]) < np.sqrt(4*np.pi) and np.abs(solution.x[1]) < np.sqrt(4*np.pi) and np.abs(solution.x[2]) < np.sqrt(4*np.pi) and np.abs(solution.x[3]) < np.sqrt(4*np.pi) and \
            np.abs(solution.x[4]) < np.sqrt(4*np.pi) and np.abs(solution.x[5]) < np.sqrt(4*np.pi) and np.abs(solution.x[6]) < np.sqrt(4*np.pi) and np.abs(solution.x[7]) < np.sqrt(4*np.pi) and \
            np.abs(solution.x[8]) < np.sqrt(4*np.pi) and np.abs(solution.x[9]) < np.sqrt(4*np.pi) and np.abs(solution.x[10]) < np.sqrt(4*np.pi) and np.abs(solution.x[11]) < np.sqrt(4*np.pi) and \
            np.abs(solution.x[12]) < np.sqrt(4*np.pi) and np.abs(solution.x[13]) < np.sqrt(4*np.pi) and np.abs(solution.x[14]) < np.sqrt(4*np.pi) and np.abs(solution.x[15]) < np.sqrt(4*np.pi) and \
            np.abs(solution.x[16]) < np.sqrt(4*np.pi) and np.abs(solution.x[17]) < np.sqrt(4*np.pi)):
            
            Omegar = np.array([[solution.x[0], solution.x[1], solution.x[2]],
                               [solution.x[3], solution.x[4], solution.x[5]],
                               [solution.x[6], solution.x[7], solution.x[8]]])
            Omegaim = np.array([[solution.x[9], solution.x[10], solution.x[11]],
                               [solution.x[12], solution.x[13], solution.x[14]],
                               [solution.x[15], solution.x[16], solution.x[17]]])

            Omega_output = Omegar + 1j*Omegaim
        
            print("Found a valid solution. Breaking code ...")
            print("\n")
            print("\u03F4 = ", Thetar + 1j*Thetaim)
            print("\n")
            print("\u03A9 = ", Omega_output)
            print("\n")
            print("mS1 = ", mS1)
            print("mS2 = ", mS2)
            print("a\u2081 = ", alpha1)
            
            break


Found a valid solution. Breaking code ...


ϴ =  [[ 9.84818270e-08-7.68228808e-02j  7.75133060e-04+6.00283086e-06j  5.43661691e-04+1.88709221e-08j]
 [ 1.72644574e-07+6.00578037e-07j -9.64592272e-05+2.23053293e-03j -2.44661906e-05+1.09655108e-06j]
 [ 1.21687342e-01+1.09210843e-06j  3.08327945e-03-1.05914818e-06j -3.35023037e-07+4.96184470e-03j]]


Ω =  [[ 3.49210194e+00-7.26285650e-01j  7.13431074e-01-1.04353915e-01j  2.59695928e-05+5.13750749e-03j]
 [ 4.87396737e-01-7.33260942e-04j  1.50006432e-03+2.25835292e-02j  1.30617854e-03+6.06737705e-03j]
 [-2.47533709e-01+2.26999804e+00j -2.00215222e-03+5.09189647e-01j -3.78064843e-04-5.55020609e-03j]]


mS1 =  2051.8835714490633
mS2 =  3671.8018631260893
a₁ =  22.378812539087725


# Checking if the point found is consistent with neutrino physics

# Let us first look at what were the inputs

In [5]:
print("Input PMNS matrix: \n\n", np.abs(VPMNS), '\n')
print("Input neutrino masses: \n\n", np.array([m1p, m2p, m3p]))

Input PMNS matrix: 

 [[0.80385893 0.5759048  0.14881023]
 [0.33789491 0.64801287 0.68257333]
 [0.4895282  0.49841044 0.71550623]] 

Input neutrino masses: 

 [5.40319834e-12 1.03454719e-11 5.18599720e-11]


# Let us recalculate the PMNS and the masses with the outputs of the run

In [6]:
#Theta was input, so use the same values
Theta_output = Thetar + 1j*Thetaim

#Omega was calculated, so we use the output values
#Omega is inside solutions, and the order of the values is the same as the order
#given in the guesses
Omegar = np.array([[solution.x[0], solution.x[1], solution.x[2]],
                   [solution.x[3], solution.x[4], solution.x[5]],
                   [solution.x[6], solution.x[7], solution.x[8]]])
Omegaim = np.array([[solution.x[9], solution.x[10], solution.x[11]],
                   [solution.x[12], solution.x[13], solution.x[14]],
                   [solution.x[15], solution.x[16], solution.x[17]]])

Omega_output = Omegar + 1j*Omegaim

#Calculate the neutrino mass matrix
Mnu11 = Neutrino_oneloop(i=0, j=0, Omega=Omega_output, Theta=Theta_output, VCKM=VCKM, v=v, a1=alpha1, md=md, mS1=mS1, mS2=mS2)
Mnu12 = Neutrino_oneloop(i=0, j=1, Omega=Omega_output, Theta=Theta_output, VCKM=VCKM, v=v, a1=alpha1, md=md, mS1=mS1, mS2=mS2)
Mnu13 = Neutrino_oneloop(i=0, j=2, Omega=Omega_output, Theta=Theta_output, VCKM=VCKM, v=v, a1=alpha1, md=md, mS1=mS1, mS2=mS2)
Mnu21 = Neutrino_oneloop(i=1, j=0, Omega=Omega_output, Theta=Theta_output, VCKM=VCKM, v=v, a1=alpha1, md=md, mS1=mS1, mS2=mS2)
Mnu22 = Neutrino_oneloop(i=1, j=1, Omega=Omega_output, Theta=Theta_output, VCKM=VCKM, v=v, a1=alpha1, md=md, mS1=mS1, mS2=mS2)
Mnu23 = Neutrino_oneloop(i=1, j=2, Omega=Omega_output, Theta=Theta_output, VCKM=VCKM, v=v, a1=alpha1, md=md, mS1=mS1, mS2=mS2)
Mnu31 = Neutrino_oneloop(i=2, j=0, Omega=Omega_output, Theta=Theta_output, VCKM=VCKM, v=v, a1=alpha1, md=md, mS1=mS1, mS2=mS2)
Mnu32 = Neutrino_oneloop(i=2, j=1, Omega=Omega_output, Theta=Theta_output, VCKM=VCKM, v=v, a1=alpha1, md=md, mS1=mS1, mS2=mS2)
Mnu33 = Neutrino_oneloop(i=2, j=2, Omega=Omega_output, Theta=Theta_output, VCKM=VCKM, v=v, a1=alpha1, md=md, mS1=mS1, mS2=mS2)
Mnu=np.array([[Mnu11, Mnu12, Mnu13],[Mnu21, Mnu22, Mnu23],[Mnu31, Mnu32, Mnu33]])

#Calculate neutrino masses and PMNS matrix
#using eigen function defined on the top of the notebook

Mnu_diag_output , UPMNS_output=eigen(Mnu)

In [7]:
print("Input PMNS matrix: \n\n", np.abs(VPMNS), '\n')
print("Output PMNS matrix: \n\n", np.abs(UPMNS_output), '\n')
print("####################################################")
print("####################################################\n")
print("Input neutrino masses: \n\n", np.array([m1p, m2p, m3p]), '\n')
print("Output neutrino masses: \n\n", Mnu_diag_output, '\n')
print("####################################################")
print("####################################################, \n")
print("Theta Yukawa matrix: \n\n", Theta_output, '\n')
print("Omega Yukawa matrix: \n\n", Omega_output, '\n')
print("Leptoquark masses: \n\n", mS1, mS2, '\n')
print("Mixing parameter: \n\n", alpha1, '\n')

Input PMNS matrix: 

 [[0.80385893 0.5759048  0.14881023]
 [0.33789491 0.64801287 0.68257333]
 [0.4895282  0.49841044 0.71550623]] 

Output PMNS matrix: 

 [[0.80385893 0.5759048  0.14881023]
 [0.33789491 0.64801287 0.68257333]
 [0.4895282  0.49841044 0.71550623]] 

####################################################
####################################################

Input neutrino masses: 

 [5.40319834e-12 1.03454719e-11 5.18599720e-11] 

Output neutrino masses: 

 [5.40319834e-12 1.03454719e-11 5.18599720e-11] 

####################################################
####################################################, 

Theta Yukawa matrix: 

 [[ 9.84818270e-08-7.68228808e-02j  7.75133060e-04+6.00283086e-06j  5.43661691e-04+1.88709221e-08j]
 [ 1.72644574e-07+6.00578037e-07j -9.64592272e-05+2.23053293e-03j -2.44661906e-05+1.09655108e-06j]
 [ 1.21687342e-01+1.09210843e-06j  3.08327945e-03-1.05914818e-06j -3.35023037e-07+4.96184470e-03j]] 

Omega Yukawa matrix: 

 [[ 3.49210194e+00-

# As one can see, we can reproduce the correct mixing and neutrino masses.